In [35]:
import librosa
import librosa.display
import numpy as np
import scipy
from scipy import signal
from IPython.display import Audio, IFrame, display

In [36]:
class Noise_removal:
    """
    Remove noise from audio based upon a clip containing only noise

    Args:
    path - The path of the source audio file.
    noise_points (array): The duration chosen to sample the noise.
    sample_rate - The sample rate using which you want to sample the audio file
    n_grad_freq (int): how many frequency channels to smooth over with the mask.
    n_grad_time (int): how many time channels to smooth over with the mask.
    n_fft (int): number audio of frames between STFT columns.
    win_length (int): Each frame of audio is windowed by `window()`. The window will be of length `win_length` and then padded with zeros to match `n_fft`..
    hop_length (int):number audio of frames between STFT columns.
    n_std_thresh (int): how many standard deviations louder than the mean dB of the noise (at each frequency level) to be considered signal
    prop_decrease (float): To what extent should you decrease noise (1 = all, 0 = none)

    Returns:
    array: The recovered signal with noise subtracted

    """
    def __init__(
        self, path, 
        noise_points,
        sample_rate=16000,
        n_grad_freq=2,
        n_grad_time=4,
        n_fft=2048,
        win_length=2048,
        hop_length=512,
        n_std_thresh=1.5,
        prop_decrease=1.0,
    ):
        self.path = path
        self.sr = sample_rate
        self.noise_start = noise_points[0]
        self.noise_end = noise_points[1]
        self.n_grad_freq = n_grad_freq
        self.n_grad_time = n_grad_time
        self.n_fft = n_fft
        self.win_length = win_length
        self.hop_length = hop_length
        self.n_std_thresh = n_std_thresh
        self.prop_decrease = prop_decrease

    def remove_noise(self):
        audio_clip,sr = librosa.load(self.path, mono=True, sr=self.sr, offset=0)
        noise_clip = audio_clip[self.noise_start*sr:self.noise_end*sr]
        # STFT over noise
        noise_stft = self._stft(noise_clip, self.n_fft, self.hop_length, self.win_length)
        noise_stft_db = self._amp_to_db(np.abs(noise_stft))  # convert to dB
        # Calculate statistics over noise
        mean_freq_noise = np.mean(noise_stft_db, axis=1)
        std_freq_noise = np.std(noise_stft_db, axis=1)
        noise_thresh = mean_freq_noise + std_freq_noise * self.n_std_thresh
        # STFT over signal
        sig_stft = self._stft(audio_clip, self.n_fft, self.hop_length, self.win_length)
        sig_stft_db = self._amp_to_db(np.abs(sig_stft))
        # Calculate value to mask dB to
        mask_gain_dB = np.min(self._amp_to_db(np.abs(sig_stft)))
        #print(noise_thresh, mask_gain_dB)
        # Create a smoothing filter for the mask in time and frequency
        smoothing_filter = np.outer(
            np.concatenate([np.linspace(0, 1, self.n_grad_freq + 1, endpoint=False), 
                            np.linspace(1, 0, self.n_grad_freq + 2),])[1:-1], 
            np.concatenate([np.linspace(0, 1, self.n_grad_time + 1, endpoint=False),
                            np.linspace(1, 0, self.n_grad_time + 2),])[1:-1],)
        smoothing_filter = smoothing_filter / np.sum(smoothing_filter)
        # calculate the threshold for each frequency/time bin
        db_thresh = np.repeat(np.reshape(noise_thresh, [1, len(mean_freq_noise)]), np.shape(sig_stft_db)[1],axis=0,).T
        # mask if the signal is above the threshold
        sig_mask = sig_stft_db < db_thresh
        # convolve the mask with a smoothing filter
        sig_mask = scipy.signal.fftconvolve(sig_mask, smoothing_filter, mode="same")
        sig_mask = sig_mask * self.prop_decrease
        # mask the signal
        sig_stft_db_masked = (sig_stft_db * (1 - sig_mask) + np.ones(np.shape(mask_gain_dB)) * mask_gain_dB * sig_mask)  # mask real
        sig_imag_masked = np.imag(sig_stft) * (1 - sig_mask)
        sig_stft_amp = (self._db_to_amp(sig_stft_db_masked) * np.sign(sig_stft)) + (1j * sig_imag_masked)
        # recover the signal
        recovered_signal = self._istft(sig_stft_amp, self.hop_length, self.win_length)
        recovered_spec = self._amp_to_db(
        np.abs(self._stft(recovered_signal, self.n_fft, self.hop_length, self.win_length)))
        return recovered_signal  
    
    def _stft(self, y, n_fft, hop_length, win_length):
        return librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)

    def _istft(self, y, hop_length, win_length):
        return librosa.istft(y, hop_length, win_length)

    def _amp_to_db(self, x):
        return librosa.core.amplitude_to_db(x, ref=1.0, amin=1e-20, top_db=80.0)

    def _db_to_amp(self, x):
        return librosa.core.db_to_amplitude(x, ref=1.0)

In [37]:
display(Audio(path,rate=16000))

In [38]:
path = 'E:\College Stuff\Semester 7\Final Year Project\Review 3\sample.wav'
sr = 16000
yg = Noise_removal(path, [5, 6], sample_rate=sr).remove_noise()

In [39]:
display(Audio(yg,rate=16000))

Note: you may need to restart the kernel to use updated packages.
